문제 : https://dacon.io/competitions/official/236046/overview/description

- CNN 모델들 정리 :  https://yeong-jin-data-blog.tistory.com/entry/%EC%82%AC%EC%A0%84%ED%95%99%EC%8A%B5-%EB%AA%A8%EB%8D%B8-CNN

- efficientNet 논문 :  https://greeksharifa.github.io/computer%20vision/2022/03/01/EfficientNet/

# import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/personal/block_pattern_classification"


/content/drive/MyDrive/personal/block_pattern_classification


In [ ]:
!pip install efficientnet_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=1c65ed9fbeb1bdb7df0869e31df96929b80b113e7324cc692734d2cc22758a1d
  Stored in directory: /root/.cache/pip/wheels/84/b9/90/25a0195cf95fb5533db96f1c77ea3f296b7cc86ae8ae48e3dc
Successfully built efficientnet_pytorch


In [ ]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models
from efficientnet_pytorch import EfficientNet

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore') 

albumentation : https://hoya012.github.io/blog/albumentation_tutorial/

# main

In [ ]:
cd /content/drive/MyDrive/personal/block_pattern_classification

/content/drive/MyDrive/personal/block_pattern_classification


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [ ]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':50,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':64,
    'SEED':41
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
df = pd.read_csv('./data/train.csv')

In [ ]:
for i in range(len(df)):
  df.loc[i, 'img_path'] = "/content/drive/MyDrive/personal/block_pattern_classification/data" + df.loc[i, 'img_path'][1:]

In [ ]:
df

,id,img_path,A,B,C,D,E,F,G,H,I,J
0,TRAIN_00000,/content/drive/MyDrive/personal/block_pattern_...,1,0,0,0,0,0,0,0,0,0
1,TRAIN_00001,/content/drive/MyDrive/personal/block_pattern_...,1,0,0,0,0,0,0,0,0,0
2,TRAIN_00002,/content/drive/MyDrive/personal/block_pattern_...,1,0,0,0,0,0,0,0,0,0
3,TRAIN_00003,/content/drive/MyDrive/personal/block_pattern_...,1,0,0,0,0,0,0,0,0,0
4,TRAIN_00004,/content/drive/MyDrive/personal/block_pattern_...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
32989,TRAIN_32989,/content/drive/MyDrive/personal/block_pattern_...,1,0,0,0,1,0,1,1,1,1
32990,TRAIN_32990,/content/drive/MyDrive/personal/block_pattern_...,1,0,0,0,0,1,1,1,1,1
32991,TRAIN_32991,/content/drive/MyDrive/personal/block_pattern_...,1,0,0,0,0,1,1,1,1,1
32992,TRAIN_32992,/content/drive/MyDrive/personal/block_pattern_...,1,0,0,0,0,1,1,1,1,1


In [ ]:
df = df.sample(frac=1)
train_len = int(len(df) * 0.8)
train_data = df[:train_len]
val_data = df[train_len:]

In [ ]:
def get_labels(df):
    return df.iloc[:,2:].values

In [ ]:
train_labels = get_labels(train_data)
val_labels = get_labels(val_data)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        try:
          image = cv2.imread(img_path)
        except:
          print(img_path)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
train_dataset = CustomDataset(train_data['img_path'].values, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_data['img_path'].values, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:

class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, adaptive=False, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, adaptive=adaptive, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups
        self.defaults.update(self.base_optimizer.defaults)

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                self.state[p]["old_p"] = p.data.clone()
                e_w = (torch.pow(p, 2) if group["adaptive"] else 1.0) * p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.data = self.state[p]["old_p"]  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        ((torch.abs(p) if group["adaptive"] else 1.0) * p.grad).norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm


In [ ]:
class MyModel(nn.Module):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__()
        self.backbone =  EfficientNet.from_pretrained('efficientnet-b4')
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = F.sigmoid(self.classifier(x))
        return x

In [ ]:
def train(model, optimizer_SAM, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_acc = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
          try:
            imgs = imgs.float().to(device)
            labels = labels.to(device)
                        
            output = model(imgs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer_SAM.first_step(zero_grad=True)

            criterion(model(imgs), labels).backward()
            optimizer_SAM.second_step(zero_grad=True)
            
            train_loss.append(loss.item())
          except:
            print("img_load_failed")
            continue
                    
        _val_loss, _val_acc = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_acc)
            
        if best_val_acc < _val_acc:
            best_val_acc = _val_acc
            best_model = model
            torch.save(best_model, "/content/drive/MyDrive/personal/block_pattern_classification/b4.pt")
    return best_model

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    val_acc = []
    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            probs = model(imgs)
            
            loss = criterion(probs, labels)
            
            probs  = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            preds = probs > 0.5
            batch_acc = (labels == preds).mean()
            
            val_acc.append(batch_acc)
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_acc = np.mean(val_acc)
    
    return _val_loss, _val_acc

In [ ]:
model = MyModel()
model.eval()
base_optimizer = torch.optim.SGD
optimizer_SAM = SAM(model.parameters(), base_optimizer, lr=CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_SAM, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer_SAM, train_loader, val_loader, scheduler, device)


Loaded pretrained weights for efficientnet-b4


  0%|          | 0/413 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.72933] Val Loss : [0.71303] Val ACC : [0.51898]


  0%|          | 0/413 [00:00<?, ?it/s]

...

  0%|          | 0/104 [00:00<?, ?it/s]

Epoch [22], Train Loss : [0.61604] Val Loss : [0.60361] Val ACC : [0.67038]


  0%|          | 0/413 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
optimizer_SAM_2 = SAM(model.parameters(), base_optimizer, lr=5e-3)
infer_model = train(model, optimizer_SAM_2, train_loader, val_loader, scheduler, device)


  0%|          | 0/413 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.59345] Val Loss : [0.56365] Val ACC : [0.70619]


  0%|          | 0/413 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

...

  0%|          | 0/413 [00:00<?, ?it/s]

img_load_failed


  0%|          | 0/104 [00:00<?, ?it/s]

/content/drive/MyDrive/personal/block_pattern_classification/data/train/TRAIN_19175.jpg


UnboundLocalError: ignored

In [ ]:
optimizer_SAM_3 = SAM(model.parameters(), base_optimizer, lr=1e-2)
infer_model = train(model, optimizer_SAM_3, train_loader, val_loader, scheduler, device)


  0%|          | 0/413 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.45119] Val Loss : [0.41958] Val ACC : [0.79857]


  0%|          | 0/413 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

...

  0%|          | 0/104 [00:00<?, ?it/s]

Epoch [5], Train Loss : [0.35742] Val Loss : [0.33449] Val ACC : [0.84881]


  0%|          | 0/413 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
optimizer_SAM_4 = SAM(model.parameters(), base_optimizer, lr=CFG["LEARNING_RATE"])
infer_model = train(model, optimizer_SAM_4, train_loader, val_loader, scheduler, device)


  0%|          | 0/413 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.32908] Val Loss : [0.31618] Val ACC : [0.85871]


  0%|          | 0/413 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

...

Epoch [31], Train Loss : [0.31376] Val Loss : [0.30178] Val ACC : [0.86650]


  0%|          | 0/413 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

Epoch [32], Train Loss : [0.31371] Val Loss : [0.30139] Val ACC : [0.86689]


  0%|          | 0/413 [00:00<?, ?it/s]

In [ ]:
test = pd.read_csv('./data/test.csv')
for i in range(len(test)):
  test.loc[i, 'img_path'] = "/content/drive/MyDrive/personal/block_pattern_classification/data" + test.loc[i, 'img_path'][1:]

In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            preds = probs > 0.5
            preds = preds.astype(int)
            predictions += preds.tolist()
    return predictions

In [ ]:
preds = inference(model, test_loader, device)


In [ ]:
submit = pd.read_csv('./data/sample_submission.csv')

In [ ]:
submit.iloc[:,1:] = preds
submit.head()

In [ ]:
submit.to_csv('./data/b4_submit_3.csv', index=False)